In [1]:
import pandas as pd
import funzioni as fx
import numpy as np

model = fx.importa_modello("../Dati_borsa/Dati_addestramento/RF_est1000/Modello_RF_est1000.joblib")


In [2]:
df = pd.read_parquet("../Dati_borsa/Dati_calcolati_scalati_approssimati_lagged/BTG.parquet")
df = fx.definizione_parametri_e_target(df, n_barre_passate=10)
X = df.drop(columns=["Target"])
y = df["Target"]
y_pred = model.predict_proba(X)
y_pred_true = y_pred[:, 1]
y_pred_true = pd.Series(y_pred_true)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    2.3s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    5.5s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:    6.0s
[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed:    6.3s finished


In [3]:

y_pred_true.name = "prob_True"
y_pred_true.index = X.index
X = pd.concat([X, y_pred_true], axis=1)


In [5]:
pos_aperta = False
tp = sl = n_azioni = prezzo_acquisto = prezzo_tot = 0
bilancio = 1000

posizioni = pd.DataFrame(columns=["Bilancio", "Direzione", "Prezzo_un", "n_azioni", "Prezzo_tot", "TP", "SL", "Open", "High", "Low", "Esito"], index=X.index)

for idx, row in X.iterrows():
    if bilancio <= 0:
        break
    print(f"\r{idx} Bilancio = {bilancio}                ", end=" ", flush=True)
    if pos_aperta == False:
        if row["prob_True"] > 0.5 and bilancio > 0:
            #COMPRA
            prezzo_acquisto = row["Open"]
            n_azioni = bilancio // prezzo_acquisto
            prezzo_tot = n_azioni * prezzo_acquisto  
            bilancio -= prezzo_tot          
            tp = prezzo_acquisto * 1.2
            sl = prezzo_acquisto * 0.95
            pos_aperta = True
            posizioni.loc[idx] = {"Bilancio": bilancio, "Direzione": "COMPRA", "Prezzo_un": prezzo_acquisto, "n_azioni": n_azioni, "Prezzo_tot": -prezzo_tot, "TP": tp, "SL": sl, "Open": row["Open"], "High": row["High"], "Low": row["Low"],}
    else:
        if row["High"] >= tp:
            prezzo_tot = tp * n_azioni
            bilancio += prezzo_tot
            pos_aperta = False
            posizioni.loc[idx] = {"Bilancio": bilancio, "Direzione": "VENDI", "Prezzo_un": tp, "n_azioni": n_azioni, "Prezzo_tot": prezzo_tot, "Esito": "Vincita", "Open": row["Open"], "High": row["High"], "Low": row["Low"]}
        elif row["Low"] <= sl:
            prezzo_tot = sl * n_azioni
            bilancio += prezzo_tot
            pos_aperta = False
            posizioni.loc[idx] = {"Bilancio": bilancio, "Direzione": "VENDI", "Prezzo_un": sl, "n_azioni": n_azioni, "Prezzo_tot": prezzo_tot, "Esito": "Perdita", "Open": row["Open"], "High": row["High"], "Low": row["Low"]}
    
posizioni.to_excel("posizioni.xlsx")
X.to_excel("X.xlsx")
print("\nfine")


3653 Bilancio = 67250.59999999998                                                                                
fine
